In [1]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup     
import pandas as pd   
import openpyxl
import math
import time  

# Step 2. 크롬 드라이버 설정 
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

#Step 3. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?: ')
print("\n")

choice = input('''2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?

1.학위논문       2.국내학술논문     3.해외학술논문     4.학술지      
5.단행본         6.공개강의         7.연구보고서     

위 장르 중 수집할 장르의 번호를 입력하세요:  ''')

print("\n")

fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\py_temp\\riss_2.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\py_temp\\riss_2.xls): ')
print("\n")    

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?: 전염병


2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?

1.학위논문       2.국내학술논문     3.해외학술논문     4.학술지      
5.단행본         6.공개강의         7.연구보고서     

위 장르 중 수집할 장르의 번호를 입력하세요:  2


3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\py_temp\riss_2.csv): riss2.csv
4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\py_temp\riss_2.xls): riss2.xls




In [2]:
#Step 4. 검색어 입력한 후 검색하여 해당 장르로 이동하기
query_url = 'http://www.riss.kr/'
driver.get(query_url)
driver.maximize_window()
time.sleep(2)

s_time = time.time( )
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

# 장르 선택하기
if choice == '1' :
    driver.find_element(By.LINK_TEXT,'학위논문').click()
elif choice =='2':
    driver.find_element(By.LINK_TEXT,'국내학술논문').click()
elif choice =='3':
    driver.find_element(By.LINK_TEXT,'해외학술논문').click()
elif choice =='4':
    driver.find_element(By.LINK_TEXT,'학술지').click()
elif choice =='5':
    driver.find_element(By.LINK_TEXT,'단행본').click()
elif choice =='6':
    driver.find_element(By.LINK_TEXT,'공개강의').click()
elif choice =='7':
    driver.find_element(By.LINK_TEXT,'연구보고서').click()

# Step 5. 크롤링 건수 추출 후 건수 입력받기
time.sleep(0.2)
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

if choice =='1' :
    total_1 = soup_1.find('div','searchBox pd')
elif choice =='2' :
    total_1 = soup_1.find('div','searchBox')
elif choice =='3' :
    total_1 = soup_1.find('div','searchBox')
elif choice =='4' :
    total_1 = soup_1.find('div','searchBox pd')
elif choice =='5' :
    total_1 = soup_1.find('div','searchBox pd')
elif choice =='6' :
    total_1 = soup_1.find('div','searchBox pd')
elif choice =='7' :
    total_1 = soup_1.find('div','searchBox pd')

total_2 = total_1.select('dl > dd > span')
total_3 = total_2[0].find('span','num').get_text()
total = total_3.replace(",","")

time.sleep(1)

#장르 이름 지정하기
if choice == '1' :
    j_name='학위논문'
elif choice =='2':
    j_name='국내학술논문'
elif choice =='3':
    j_name='해외학술논문'
elif choice =='4':
    j_name='학술지'
elif choice =='5':
    j_name='단행본'
elif choice =='6':
    j_name='공개강의'
elif choice =='7':
    j_name='연구보고서'

if total =='0' :
    print("%s 키워드로 %s 부분에서 검색된 자료의 건수는 총 0건 입니다" %(query_txt,j_name) )
    print("검색된 결과 건수가 0건이므로 수집할 데이터가 없어서 종료합니다")
else :
    print("%s 키워드로 %s 부분에서 검색된 자료의 건수는 총 %s 건 입니다"%(query_txt,j_name,total))
    cnt = int(input('이 중에서 크롤링 할 건수는 몇건입니까?: '))
    page_cnt = math.ceil(cnt / 10) 

    # 국내학술논문 카테고리의 정보 수집하기
    if choice =='2' and cnt > 0:
        print("%s 키워드로 국내학술논문을 검색하여 총 %s 건 중 %s건의 정보를 수집하겠습니다" %(query_txt,total,cnt))

        no2=[]           # 게시글 번호 컬럼
        title2=[ ]       # 게시글 제목 컬럼
        author2=[]       # 논문 저자 컬럼
        company2=[ ]     # 소속 기관 컬럼
        date2=[ ]        # 게시글 날짜 컬럼
        suksa2=[ ]       # 국내석사 컬럼
        contents2=[]     # 초록내용
        full_url2=[]     # 논문 원본 URL

        no = 1           # 게시글 번호 초기값
        k = 0   # 페이지 번호 넘기기 위해 사용하는 변수
        #xpath_no = 3

        for x in range(1,page_cnt+1) :
            print("\n")
            print("%s 페이지 내용 수집 시작합니다 =======================" %x)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            content_list = soup.find('div','srchResultListW').find_all('li')

            for i in content_list:

                # 2. 게시글 제목
                try:
                    title=i.find('div','cont').find('p','title').get_text().strip()
                except :
                    continue 
                else :

                    # 1.게시글 번호
                    print("\n")
                    print("%s 번째 정보를 추출하고 있습니다============" %no)
                    no2.append(no)
                    print("1.번호 : %s" %no)

                    title2.append(title.strip())
                    print("2.제목 : %s" %title.strip())

                    # 3. 작성자
                    try :
                        author=i.find('p','etc').find('span','writer').get_text().strip()
                    except :
                        author = '작성자가 없습니다'
                        print("3.작성자 : %s" %author.strip())
                        author2.append(author.strip())
                    else :
                        author2.append(author.strip())
                        print("3.작성자 : %s" %author.strip())

                    # 4. 소속기관
                    try :
                        company=i.find('p','etc').find('span','assigned').get_text().strip()
                    except :
                        company='소속 기관이 없습니다'
                        company2.append(company.strip())
                        print("4.소속기관 : %s" %company.strip())
                    else :
                        company2.append(company.strip())
                        print("4.소속기관 : %s" %company.strip())

                    # 5. 발표날짜
                    try :
                        date_1 =i.find('p','etc').find_all('span')
                        date_2 = date_1[2].get_text().strip()
                    except :
                        date_2='발표날짜가 없습니다'
                        date2.append(date_2)
                        print("5.발표년도 : %s" %date_2)
                    else :
                        date2.append(date_2)
                        print("5.발표년도 : %s" %date_2)

                    # 6.실린 논문집
                    try :
                        suksa_1 =i.find('p','etc').find_all('span')
                        suksa_2 = suksa_1[3].get_text().strip()
                    except :
                        suksa_2='논문집이 없습니다'
                        suksa2.append(suksa_2)
                        print("6.논문집/자료집 : %s" %suksa_2)
                    else :
                        suksa2.append(suksa_2)
                        print("6.논문집/자료집 : %s" %suksa_2)

                    # 7.논문 url 주소       
                    url_1 = i.find('p','title').find('a')['href']
                    full_url = 'http://www.riss.kr'+url_1

                    full_url2.append(full_url)
                    print('7.논문 URL 주소:' , full_url)

                    time.sleep(2)

                    #news_no += 1
                    no += 1

                    if no > cnt :
                        break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

            x += 1

            if no >= cnt :
                break
            
        try :
            driver.find_element(By.LINK_TEXT,'%s' %x).click() # 다음 페이지번호 클릭
        except :
            driver.find_element(By.LINK_TEXT,'다음 페이지로').click()

# Step 7. 출력 결과 저장하기
df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(author2)
df['소속(발행)기관']=pd.Series(company2)
df['날짜']=pd.Series(date2)
df['학위(논문일경우)']=pd.Series(suksa2)
df['초록(논문일경우)']=pd.Series(contents2)
df['자료URL주소']=pd.Series(full_url2)

# xls / csv 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8",engine='openpyxl')
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("=" *80)

driver.close( )

전염병 키워드로 국내학술논문 부분에서 검색된 자료의 건수는 총 3019 건 입니다
이 중에서 크롤링 할 건수는 몇건입니까?: 20
전염병 키워드로 국내학술논문을 검색하여 총 3019 건 중 20건의 정보를 수집하겠습니다


1 페이지 내용 수집 시작합니다 =======================


1 번째 정보를 추출하고 있습니다============
1.번호 : 1
2.제목 : 傳染病과 季節的 變化 : 特히 夏期傳染病을 中心으로
3.작성자 : 金仁達
4.소속기관 : 대한의학협회
5.발표년도 : 1961
6.논문집/자료집 : 대한의학협회지
7.논문 URL 주소: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=748adc6a61d06600ffe0bdc3ef48d419&keyword=전염병


2 번째 정보를 추출하고 있습니다============
1.번호 : 2
2.제목 : Session 1. 최근 국제적 이슈가 되고 있는 호흡기 인수공통전염병: Recent Hot Issues in Respiratory : Sl-2 ; H5N8 Avian Influenza Virus
3.작성자 : 이윤정 ( Youn Jeong Lee )
4.소속기관 : 대한인수공통전염병학회
5.발표년도 : 2014
6.논문집/자료집 : 창립총회 및 학술대회 초록집
7.논문 URL 주소: http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a43eea44f7859e1bd18150b21a227875&keyword=전염병


3 번째 정보를 추출하고 있습니다============
1.번호 : 3
2.제목 : Session 2. 곤충매개 인수공통전염병: Vectorborne Zoonotic Diseases : S2-2 ; Severe Fever with Thrombocytopenia Synd



크롤링을 요청한 총 20 건 중에서 20 건의 데이터를 수집 완료 했습니다
총 소요시간은 62.8 초 입니다 


C:\Users\shjoo\AppData\Local\Temp\ipykernel_11684\2180119826.py:204: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['초록(논문일경우)']=pd.Series(contents2)
